# Feed Forward Neural Network

In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
INPUT_SIZE = 28 * 28
HIDDEN_SIZE = 500
NUM_CLASSES = 10
NUM_EPOCHS = 5
BATCH_SIZE = 100
LEARNING_RATE = 0.001

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='../data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='../data',
    train=False,
    transform=transforms.ToTensor()
)


# Data loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [6]:
class NeuralNet(nn.Module):
    """
    Fully connected neural network with one hidden layer
    """

    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
# Compile the model
model = NeuralNet(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [10]:
# Train the model
total_step = len(train_loader)

for epoch in tqdm(range(NUM_EPOCHS), desc='Epochs', position=0, leave=True):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch [1/5], Step [100/600], Loss: 0.0236
Epoch [1/5], Step [200/600], Loss: 0.0350
Epoch [1/5], Step [300/600], Loss: 0.0220
Epoch [1/5], Step [400/600], Loss: 0.0458
Epoch [1/5], Step [500/600], Loss: 0.0092
Epoch [1/5], Step [600/600], Loss: 0.0592
Epoch [2/5], Step [100/600], Loss: 0.0060
Epoch [2/5], Step [200/600], Loss: 0.0193
Epoch [2/5], Step [300/600], Loss: 0.0084
Epoch [2/5], Step [400/600], Loss: 0.0235
Epoch [2/5], Step [500/600], Loss: 0.0281
Epoch [2/5], Step [600/600], Loss: 0.0292
Epoch [3/5], Step [100/600], Loss: 0.0092
Epoch [3/5], Step [200/600], Loss: 0.0274
Epoch [3/5], Step [300/600], Loss: 0.0199
Epoch [3/5], Step [400/600], Loss: 0.0121
Epoch [3/5], Step [500/600], Loss: 0.0107
Epoch [3/5], Step [600/600], Loss: 0.0540
Epoch [4/5], Step [100/600], Loss: 0.0108
Epoch [4/5], Step [200/600], Loss: 0.0393
Epoch [4/5], Step [300/600], Loss: 0.0219
Epoch [4/5], Step [400/600], Loss: 0.0270
Epoch [4/5], Step [500/600], Loss: 0.0302
Epoch [4/5], Step [600/600], Loss:

In [16]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        # Returns the maximum value of each row of the input tensor in the given 
        # dimension dim, the second return value is the index location of each 
        # maximum value found (argmax).
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

Accuracy of the network on the 10000 test images: 98.07%


In [18]:
# Save the model checkpoint
torch.save(model.state_dict(), 'ff-model.ckpt')